# Data-Centric NLP 대회: 주제 분류 프로젝트

## Load Libraries

In [ ]:
%%bash
pip install tqdm
pip install numpy
pip install pandas
pip install transformers==4.26.0
pip install sentencepiece==0.1.96
pip install evaluate==0.4.0
pip install accelerate
pip install scikit-learn
pip install ipywidgets
pip install protobuf==3.15.8
pip install torch==1.11.0

In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

## Set Hyperparameters

In [2]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [3]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [4]:
BASE_DIR = "/data/ephemeral/level2-nlp-datacentric-nlp-06/"
DATA_DIR = os.path.join(BASE_DIR, 'data/')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output/')
LOG_DIR = os.path.join(BASE_DIR, 'logs/')
CACHE_DIR = os.path.join(BASE_DIR, 'cache/')

## Load Tokenizer and Model

In [5]:
model_name = 'klue/bert-base'
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7, cache_dir=CACHE_DIR).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=CACHE_DIR)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initial

## Define Dataset

In [6]:
data = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
dataset_train, dataset_valid = train_test_split(data, test_size=0.3, stratify=data['target'],random_state=SEED)

In [7]:
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []
        self.labels = []
        
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),  
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }
    
    def __len__(self):
        return len(self.labels)

In [8]:
data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define Metric

In [10]:
f1 = evaluate.load('f1')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

## Train Model

### wandb & mecab setting

In [ ]:
!pip install wandb

In [ ]:
%%bash
su -
apt-get update
apt-get install sudo -y
sudo apt install make

sudo apt-get install g++ openjdk-8-jdk

In [ ]:
%cd ..

In [ ]:
%%bash
cd mecab-0.996-ko-0.9.2
./configure
make
make check
sudo make install

In [ ]:
%cd ..

In [ ]:
%%bash
cd mecab-ko-dic-2.1.1-20180720
./configure
sudo ldconfig
make
sudo make install

In [ ]:
%%bash
pip install konlpy
pip install mecab-python3
pip install JPype1-py3

### train

In [11]:
from konlpy.tag import Mecab
mecab = Mecab()

In [12]:
import wandb
import os

## for wandb setting
os.environ["WANDB_API_KEY"] = "6f6f03437c7053a9ea262a0afa0edb5694d48a1d"
os.environ["WANDB_PROJECT"] = "clf_HYW"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

In [13]:
wandb.init(name="baseline_code")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hwyewon (be-our-friend). Use `wandb login --relogin` to force relogin


In [17]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    logging_dir=LOG_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    logging_strategy='steps',
    evaluation_strategy='steps',
    save_strategy='steps',
    logging_steps=500,
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED,
    report_to="wandb",
    run_name="baseline_code"
)

PyTorch: setting up devices


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4900
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2450
  Number of trainable parameters = 110622727
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,F1
500,0.416900,0.826161,0.836535
1000,0.680400,0.728455,0.842384
1500,0.600100,0.802052,0.849418
2000,0.425800,0.799729,0.849380


***** Running Evaluation *****
  Num examples = 2100
  Batch size = 4
Saving model checkpoint to /data/ephemeral/level2-nlp-datacentric-nlp-06/output/checkpoint-500
Configuration saved in /data/ephemeral/level2-nlp-datacentric-nlp-06/output/checkpoint-500/config.json
Model weights saved in /data/ephemeral/level2-nlp-datacentric-nlp-06/output/checkpoint-500/pytorch_model.bin
Logging checkpoint artifacts in checkpoint-500. ...
wandb: Adding directory to artifact (/data/ephemeral/level2-nlp-datacentric-nlp-06/output/checkpoint-500)... Done. 8.2s
***** Running Evaluation *****
  Num examples = 2100
  Batch size = 4
Saving model checkpoint to /data/ephemeral/level2-nlp-datacentric-nlp-06/output/checkpoint-1000
Configuration saved in /data/ephemeral/level2-nlp-datacentric-nlp-06/output/checkpoint-1000/config.json
Model weights saved in /data/ephemeral/level2-nlp-datacentric-nlp-06/output/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [/data/ephemeral/level2-nlp-datacentric-nlp-0

TrainOutput(global_step=2450, training_loss=0.5152296603455836, metrics={'train_runtime': 525.993, 'train_samples_per_second': 18.631, 'train_steps_per_second': 4.658, 'total_flos': 2578604098560000.0, 'train_loss': 0.5152296603455836, 'epoch': 2.0})

## Evaluate Model

### validation (total)

In [20]:
model.eval()
preds = []
for idx, sample in tqdm(dataset_valid.iterrows()):
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        preds.extend(pred)

2100it [00:19, 108.79it/s]


In [21]:
f1 = evaluate.load('f1')
labels = dataset_valid['target'].values

f1.compute(predictions=preds, references=labels, average='macro')

{'f1': 0.8494182653265189}

## inference

In [22]:
dataset_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [23]:
model.eval()
preds = []
for idx, sample in tqdm(dataset_test.iterrows()):
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        preds.extend(pred)

47785it [07:14, 109.95it/s]


In [24]:
dataset_test['target'] = preds
dataset_test.to_csv(os.path.join(BASE_DIR, 'output.csv'), index=False)

In [25]:
dataset_test.head()

,ID,text,url,date,target
0,ynat-v1_dev_00000,제임스 부상 레이커스 성탄매치서 골든스테이트에 완승종합,https://sports.news.naver.com/news.nhn?oid=001...,2018.12.26 15:16,5
1,ynat-v1_dev_00001,프랑스 극우정치인 르펜 노란 조끼 덕에 승승장구,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.17. 오후 8:04,4
2,ynat-v1_dev_00002,대통령개헌안 ⑥토지공개념 명시…개발이익환수·부동산과세 강화 전망종합,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.03.21. 오후 1:09,6
3,ynat-v1_dev_00003,의사 살해 환자 경찰서 나와 법원으로,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.02. 오후 2:47,2
4,ynat-v1_dev_00004,이란 최고지도자 유럽 맹비난…핵합의 미이행 뻔뻔하고 오만,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.16. 오후 8:11,4
